In [1]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *

# Creating FASTQ files
* 2 samples each time. you should only specify their names and the path for the joint CSV.
* Should be a folder with CSV mapping the barcodes for both samples.
* The output is a bash file which you can run and create the complete FASTQ files - it will be located in 'output_bash_file', and the output of it will be located in '/storage/md_keren/shitay/10X/fastq/'
* you can use this script to create later condor_submit/job files that will be located in '/srv01/technion/shitay/scripts/FASTQ_scripts/', run them after creating the complete FASTQ file.

In [19]:
sample1_name = 'CM29'
sample2_name = 'CM30'

# important paths
PATH = fr'/storage/md_keren/shitay/10X/Drive_row_files/{sample1_name}_{sample2_name}'
output_bash_file = fr'/storage/md_keren/shitay/10X/sh_FASTQ_builders/combine_fastq_{sample1_name}_{sample2_name}.sh' ## the path of the .sh file that will be created.
FASTQ_output_path = r'/storage/md_keren/shitay/10X/FASTQ' ## the output path of the final FASTQ file after you run the .sh file that build the fastq

print(f'PATH: \n{PATH}')
print(f'output_bash_file: \n{output_bash_file}')
print(f'FASTQ_output_path: \n{FASTQ_output_path}')

PATH: 
/storage/md_keren/shitay/10X/Drive_row_files/CM29_CM30
output_bash_file: 
/storage/md_keren/shitay/10X/sh_FASTQ_builders/combine_fastq_CM29_CM30.sh
FASTQ_output_path: 
/storage/md_keren/shitay/10X/FASTQ


In [81]:
###  automatic find other important files:

CSV_PATH = join(PATH, fr'{sample1_name}_{sample2_name}.csv') ## csv path
SN0_folder = os.listdir(join(PATH, 'pkgs'))[0]
SN0_PATH = join(PATH, fr'pkgs/{SN0_folder}')
# It is a unique prefix that each of the row files contains. 
# look over the folder '/storage/md_keren/shitay/10X/M149_M150/pkgs/SN0217504/',
# the prefix there for example is '_HVM2VBGXG.' 
# prefix = '_HVM2VBGXG.' 
prefix = [tt for tt in os.listdir(SN0_PATH) if len(tt)>2 and tt[1]=='_'][0][1:12]

print(f'CSV_PATH: {CSV_PATH}')
print(f'SN0_folder: {SN0_folder}')
print(f'SN0_PATH: {SN0_PATH}')
print(f'prefix: {prefix}')

CSV_PATH: /storage/md_keren/shitay/10X/Drive_row_files/M182_M183/M182_M183.csv
SN0_folder: SN0240954
SN0_PATH: /storage/md_keren/shitay/10X/Drive_row_files/M182_M183/pkgs/SN0240954
prefix: _HL7YCBGXK.


In [82]:
df = pd.read_csv(CSV_PATH)[['SampleName', 'IndexBarcode1']]
values = df.values 
sample1_values = [[s[0],s[1]] for s in values if sample1_name in s[0]]
sample1_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample1_values}
sample2_values = [[s[0],s[1]] for s in values if sample2_name in s[0]]
sample2_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample2_values}


y = [[1, 2],[3, 4]]
[z for x in y for z in x]
sample1_values

{1: 'TTCGCCCT', 2: 'GGATGGGC', 3: 'AATCAATG', 4: 'CCGATTAA'}

In [83]:
def synthesize_command_for_sample(barcode_values, sample_name):
    n_of_channels = 4
    n_of_directions = 2
    n_of_barcodes = len(barcode_values)
    lines = []
    lines.append(f'mkdir {FASTQ_output_path}/{sample_name}')
    lines.append(f'cd {SN0_PATH}')
    for direction in range(1, 3):
        line = 'cat '
        for channel in range(1, n_of_channels+1):
            for barcode_idx in range(1, n_of_barcodes+1):
                barcode = barcode_values[barcode_idx]
                line += str(channel)+prefix+str(channel)+'.'+barcode+'.unmapped.'+str(direction)+ '.fastq.gz '
        line += f'> {FASTQ_output_path}/{sample_name}/{sample_name}_S1_L001_R{direction}_001.fastq.gz'
        lines.append(line)      
    return lines

res = synthesize_command_for_sample(sample1_values, sample1_name) + synthesize_command_for_sample(sample2_values, sample2_name)
res

['mkdir /storage/md_keren/shitay/10X/FASTQ/M182',
 'cd /storage/md_keren/shitay/10X/Drive_row_files/M182_M183/pkgs/SN0240954',
 'cat 1_HL7YCBGXK.1.TTCGCCCT.unmapped.1.fastq.gz 1_HL7YCBGXK.1.GGATGGGC.unmapped.1.fastq.gz 1_HL7YCBGXK.1.AATCAATG.unmapped.1.fastq.gz 1_HL7YCBGXK.1.CCGATTAA.unmapped.1.fastq.gz 2_HL7YCBGXK.2.TTCGCCCT.unmapped.1.fastq.gz 2_HL7YCBGXK.2.GGATGGGC.unmapped.1.fastq.gz 2_HL7YCBGXK.2.AATCAATG.unmapped.1.fastq.gz 2_HL7YCBGXK.2.CCGATTAA.unmapped.1.fastq.gz 3_HL7YCBGXK.3.TTCGCCCT.unmapped.1.fastq.gz 3_HL7YCBGXK.3.GGATGGGC.unmapped.1.fastq.gz 3_HL7YCBGXK.3.AATCAATG.unmapped.1.fastq.gz 3_HL7YCBGXK.3.CCGATTAA.unmapped.1.fastq.gz 4_HL7YCBGXK.4.TTCGCCCT.unmapped.1.fastq.gz 4_HL7YCBGXK.4.GGATGGGC.unmapped.1.fastq.gz 4_HL7YCBGXK.4.AATCAATG.unmapped.1.fastq.gz 4_HL7YCBGXK.4.CCGATTAA.unmapped.1.fastq.gz > /storage/md_keren/shitay/10X/FASTQ/M182/M182_S1_L001_R1_001.fastq.gz',
 'cat 1_HL7YCBGXK.1.TTCGCCCT.unmapped.2.fastq.gz 1_HL7YCBGXK.1.GGATGGGC.unmapped.2.fastq.gz 1_HL7YCBGXK.1.

### Creating a bash file which you can run and create the complete FASTQ files - it will be located in 'output_bash_file', and the output of it will be located in '/storage/md_keren/shitay/10X/fastq/'

In [84]:
# output_bash_file
with open(output_bash_file, "w") as text_file:
    for line in res:
        text_file.write(line)
        text_file.write('\n')

In [85]:
print('In order to run the BASH file (builder) and create the FASTQ file, run:')
print(f'sh {output_bash_file}')

In order to run the BASH file (builder) and create the FASTQ file, run:
sh /storage/md_keren/shitay/10X/sh_FASTQ_builders/combine_fastq_M182_M183.sh


# Run a job with Condor

### you can run these lines to create condor_submit/job files that will be located in '/srv01/technion/shitay/scripts/FASTQ_scripts/', run them after creating the complete FASTQ file.

In [20]:
def create_submit_file(sample_name):
    submit_content = f'executable   = run_{sample_name}.sh\n'
    submit_content += 'arguments    = $(Process)\n'

    submit_content += f'error   = /storage/md_keren/shitay/jobs/errors/{sample_name}.txt\n'
    submit_content += f'output  = /storage/md_keren/shitay/jobs/out/{sample_name}.txt\n'
    submit_content += f'log     = /storage/md_keren/shitay/jobs/logs/{sample_name}.txt\n'

    submit_content += 'request_cpus   = 10\n'
    submit_content += 'request_memory = 32000\n'

    submit_content += 'should_transfer_files = no\n'
    submit_content += 'queue 1'
    
    
    path = fr'/srv01/technion/shitay/scripts/FASTQ_scripts/submit_{sample_name}.submit'
    with open(path, 'w') as file:
        file.write(submit_content)
    
create_submit_file(sample1_name)
create_submit_file(sample2_name)

In [21]:
def create_bash_file(sample_name):

    sh_content = '#!/bin/bash\n'

    sh_content += f'CELLRANGER_PATH=/Local/md_keren/cellranger-3.1.0/\n'
    sh_content += f'TRANSCRIPTOME_PATH=/storage/md_keren/CellRangerRefData/refdata-cellranger-hg19-3.0.0\n'
    sh_content += f'FASTQ_PATH=/storage/md_keren/shitay/10X/FASTQ/{sample_name}/\n'
    sh_content += f'ID={sample_name}\n'
    sh_content += f'SAMPLE={sample_name}\n'
    sh_content += f'MEMPERCORE=10\n'
    sh_content += f'mkdir /storage/`id -gn`/`id -un`/10X/Execution/{sample_name}\n'
    sh_content += f'cd /storage/`id -gn`/`id -un`/10X/Execution/{sample_name}\n'
    sh_content += '${CELLRANGER_PATH}/cellranger count --id=${ID} --sample=${SAMPLE} --fastqs=${FASTQ_PATH} --localmem=32 --localcores=8 --jobmode=local --transcriptome=${TRANSCRIPTOME_PATH} --chemistry=auto'
    
    path = fr'/srv01/technion/shitay/scripts/FASTQ_scripts/run_{sample_name}.sh'
    with open(path, 'w') as file:
        file.write(sh_content)
    
create_bash_file(sample1_name)
create_bash_file(sample2_name)
print('Please notice that you will first need to run chmod u+x bash_file_name in order to submit the job:')
print(f'cd /srv01/technion/shitay/scripts/FASTQ_scripts')
print(f'chmod u+x '+f'run_{sample1_name}.sh')
print(f'chmod u+x '+f'run_{sample2_name}.sh')
print()
print('Then you can run:')
print(f'condor_submit '+f'submit_{sample1_name}.submit')
print(f'condor_submit '+f'submit_{sample2_name}.submit')
print(f'condor_q')

Please notice that you will first need to run chmod u+x bash_file_name in order to submit the job:
cd /srv01/technion/shitay/scripts/FASTQ_scripts
chmod u+x run_CM29.sh
chmod u+x run_CM30.sh

Then you can run:
condor_submit submit_CM29.submit
condor_submit submit_CM30.submit
condor_q


# Debug

In [34]:
def stanga(x, y):
    stanga_rec(x, y, 0, 0 ,'\n0:0 ')
    

def stanga_rec(x, y, i, j, txt):
    if x==0 and y==0:
        print(txt)
        return
    
    if x>=1:
        txt2 = txt + f'{i+1}:{j} '
        stanga_rec(x-1, y, i+1, j, txt2)
    if x>=2:
        txt3 = txt + f'{i+2}:{j} '
        stanga_rec(x-2, y, i+2, j, txt3)
    if y>=1:
        txt4 = txt + f'{i}:{j+1} '
        stanga_rec(x, y-1, i, j+1, txt4)
    if y>=2:
        txt5 = txt + f'{i}:{j+2}'
        stanga_rec(x, y-2, i, j+2, txt5)

stanga(2,2)


0:0 1:0 2:0 2:1 2:2 

0:0 1:0 2:0 2:2

0:0 1:0 1:1 2:1 2:2 

0:0 1:0 1:1 1:2 2:2 

0:0 1:0 1:22:2 

0:0 2:0 2:1 2:2 

0:0 2:0 2:2

0:0 0:1 1:1 2:1 2:2 

0:0 0:1 1:1 1:2 2:2 

0:0 0:1 2:1 2:2 

0:0 0:1 0:2 1:2 2:2 

0:0 0:1 0:2 2:2 

0:0 0:21:2 2:2 

0:0 0:22:2 
